## **Fundamental Factor Analysis in U.S. Equity Market**

### Objective
This project replicates the core methodology of *“Fundamental Indexation”*,
a long-only “smart beta” strategy that constructs portfolios weighted by fundamental metrics
such as Book Value, Income, Sales, and Dividends.

The goal is to recreate the key findings presented in **Table 1** and **Table 2** of the paper,
with two key adjustments:
1. Apply consistent data treatment and windsorization using CRSP–Compustat merged data.
2. Extend the analysis to an **out-of-sample test (2005–2024)** to evaluate the persistence of alpha.

### Data
- Source: CRSP and Compustat monthly data (1962–2024)
- Frequency: Monthly returns
- Variables: Market cap, SEQ, IB, DVT, SALE, etc.

### Methods
1. Data cleaning and variable construction  
2. Portfolio formation (top 1000 stocks by accounting variable)  
3. CAPM and Fama–French regression (in-sample & out-of-sample)  
4. Interpretation of α, β, and t-stats

### Results Summary
- CAPM explains early-period returns well (α ≈ 0)
- FF3 improves fit but α remains insignificant post-2005
- Fundamental portfolios are primarily value-tilted (β_HML > 0)

### Tools
Python (pandas, numpy, statsmodels, matplotlib)


In [3]:
import pandas as pd
import numpy as np

stocks = pd.read_feather('crsp_monthly_stocks_HW4.feather')
stocks

,PERMNO,DATE,SHRCD,EXCHCD,SICCD,PRC,VOL,RET,SPREAD,RETX,SHROUT
0,10000,1986-01-31,10,3,3990,-4.37500,1771.0,NaN,0.25000,NaN,3680.0
1,10000,1986-02-28,10,3,3990,-3.25000,828.0,-0.257143,0.25000,-0.257143,3680.0
2,10000,1986-03-31,10,3,3990,-4.43750,1078.0,0.365385,0.12500,0.365385,3680.0
3,10000,1986-04-30,10,3,3990,-4.00000,957.0,-0.098592,0.25000,-0.098592,3793.0
4,10000,1986-05-30,10,3,3990,-3.10938,1074.0,-0.222656,0.09375,-0.222656,3793.0
...,...,...,...,...,...,...,...,...,...,...,...
3892096,93436,2024-08-30,11,3,9999,214.11000,16108365.0,-0.077390,NaN,-0.077390,3194640.0
3892097,93436,2024-09-30,11,3,9999,261.63000,16042065.0,0.221942,NaN,0.221942,3207000.0
3892098,93436,2024-10-31,11,3,9999,249.85001,19014312.0,-0.045025,NaN,-0.045025,3210060.0
3892099,93436,2024-11-29,11,3,9999,345.16000,20821313.0,0.381469,NaN,0.381469,3210060.0


## **Cleaning the crsp data**

1. Shift the date so that it is always the last day of the month, rather than the last trading day.  This will make it easier to merge in with the Compustat dataset.
2. Take the absolute value of the closing price.  For shares that don't trade, CRSP sets the price equal to the closing bid-ask midpoint, but it makes the price negative as a warning about this.
3. Define market value (MV) as the product of shares outstanding and closing price and only keep market cap values that are > 0
4. Select only stocks with share codes of 10 or 11. This means that we will be downloading common equity and not other securities (ETFs, REITS, etc.).
5. Drop variables that we won't be using for this exercise to make the dataframe easier to display.
6. Set the index to PERMNO/DATE.
7. Sort by the index.

In [4]:
from pandas.tseries.offsets import MonthEnd
#1
stocks['DATE'] = stocks['DATE'] + MonthEnd(0)
#2.
stocks['PRC']  = np.abs(stocks['PRC'])
#3.
stocks['Market Cap'] = stocks['SHROUT']*stocks['PRC']
stocks = stocks[stocks['Market Cap']>0].copy()
#4.
stocks = stocks[stocks['SHRCD'].isin([10,11])].copy()
#5.
stocks.drop(['SHROUT','SHRCD','EXCHCD','SICCD','PRC','VOL','SPREAD'], axis=1, inplace=True)
#6.
stocks.set_index(['PERMNO','DATE'], inplace=True)
#7.
stocks.sort_index(level=['PERMNO','DATE'], inplace=True)

stocks.head()

RET      RETX   Market Cap
PERMNO DATE                                       
10000  1986-01-31       NaN       NaN  16100.00000
       1986-02-28 -0.257143 -0.257143  11960.00000
       1986-03-31  0.365385  0.365385  16330.00000
       1986-04-30 -0.098592 -0.098592  15172.00000
       1986-05-31 -0.222656 -0.222656  11793.87834

In [5]:
cstat = pd.read_feather('compustat_annual_HW4.feather')
cstat

,LPERMNO,DATADATE,AT,CEQ,CHE,LT,PSTK,SEQ,DVT,IB,SALE,CAPX
0,10000,1986-10-31,2.115,0.418,0.348,1.697,0.0,0.418,0.000,-0.730,1.026,0.240
1,10001,1986-06-30,12.242,5.432,0.746,6.810,0.0,5.432,0.365,0.669,21.460,0.551
2,10001,1987-06-30,11.771,5.369,0.729,6.402,0.0,5.369,0.416,0.312,16.621,0.513
3,10001,1988-06-30,11.735,5.512,0.744,6.223,0.0,5.512,0.427,0.542,16.978,0.240
4,10001,1989-06-30,18.565,6.321,1.177,12.244,0.0,6.321,0.459,1.208,22.910,0.444
...,...,...,...,...,...,...,...,...,...,...,...,...
332829,93436,2020-12-31,52148.000,22225.000,19622.000,28469.000,0.0,22225.000,0.000,721.000,31536.000,3232.000
332830,93436,2021-12-31,62131.000,30189.000,18052.000,30548.000,0.0,30189.000,0.000,5519.000,53823.000,6514.000
332831,93436,2022-12-31,82338.000,44704.000,22479.000,36440.000,0.0,44704.000,0.000,12556.000,81462.000,7163.000
332832,93436,2023-12-31,106618.000,62634.000,29637.000,43009.000,0.0,62634.000,0.000,14997.000,96773.000,8899.000


In [6]:
cstat.rename(columns={"LPERMNO":"PERMNO"}, inplace=True)
# Shift the date so that it is always the last day of the month, rather than the last trading day.
cstat['DATE'] = cstat['DATADATE'] + MonthEnd(0)

cstat.set_index(['PERMNO','DATE'], inplace=True)

cstat.sort_index(inplace=True)
cstat

DATADATE          AT        CEQ        CHE         LT  \
PERMNO DATE                                                                 
10000  1986-10-31 1986-10-31       2.115      0.418      0.348      1.697   
10001  1986-06-30 1986-06-30      12.242      5.432      0.746      6.810   
       1987-06-30 1987-06-30      11.771      5.369      0.729      6.402   
       1988-06-30 1988-06-30      11.735      5.512      0.744      6.223   
       1989-06-30 1989-06-30      18.565      6.321      1.177     12.244   
...                      ...         ...        ...        ...        ...   
93436  2020-12-31 2020-12-31   52148.000  22225.000  19622.000  28469.000   
       2021-12-31 2021-12-31   62131.000  30189.000  18052.000  30548.000   
       2022-12-31 2022-12-31   82338.000  44704.000  22479.000  36440.000   
       2023-12-31 2023-12-31  106618.000  62634.000  29637.000  43009.000   
       2024-12-31 2024-12-31  122070.000  72913.000  37057.000  48390.000   

                   PSTK        SEQ    DVT         IB       SALE       CAPX  
PERMNO DATE                                                                 
10000  1986-10-31   0.0      0.418  0.000     -0.730      1.026      0.240  
10001  1986-06-30   0.0      5.432  0.365      0.669     21.460      0.551  
       1987-06-30   0.0      5.369  0.416      0.312     16.621      0.513  
       1988-06-30   0.0      5.512  0.427      0.542     16.978      0.240  
       1989-06-30   0.0      6.321  0.459      1.208     22.910      0.444  
...                 ...        ...    ...        ...        ...        ...  
93436  2020-12-31   0.0  22225.000  0.000    721.000  31536.000   3232.000  
       2021-12-31   0.0  30189.000  0.000   5519.000  53823.000   6514.000  
       2022-12-31   0.0  44704.000  0.000  12556.000  81462.000   7163.000  
       2023-12-31   0.0  62634.000  0.000  14997.000  96773.000   8899.000  
       2024-12-31   0.0  72913.000  0.000   7091.000  97690.000  11342.000  

[332834 rows x 11 columns]

Now let's get rid of duplicates in `PERMNO` and `DATE` and keep only the variables that we need to construct 'SEQ','IB','DVT','SALE'

In [7]:

cstat  = cstat.sort_values(by = ['PERMNO','DATE','IB'], ascending = [True, True, False])
cstat  = cstat.groupby(['PERMNO','DATE']).head(1)
cstat = cstat[['SEQ','IB','DVT','SALE']].copy()
cstat

SEQ         IB    DVT       SALE
PERMNO DATE                                              
10000  1986-10-31      0.418     -0.730  0.000      1.026
10001  1986-06-30      5.432      0.669  0.365     21.460
       1987-06-30      5.369      0.312  0.416     16.621
       1988-06-30      5.512      0.542  0.427     16.978
       1989-06-30      6.321      1.208  0.459     22.910
...                      ...        ...    ...        ...
93436  2020-12-31  22225.000    721.000  0.000  31536.000
       2021-12-31  30189.000   5519.000  0.000  53823.000
       2022-12-31  44704.000  12556.000  0.000  81462.000
       2023-12-31  62634.000  14997.000  0.000  96773.000
       2024-12-31  72913.000   7091.000  0.000  97690.000

[332834 rows x 4 columns]

In [8]:
ratios_df = stocks[['Market Cap']].merge(cstat, how='left', left_index=True, right_index=True)
ratios_df

Market Cap      SEQ      IB  DVT     SALE
PERMNO DATE                                                   
10000  1986-01-31  1.610000e+04      NaN     NaN  NaN      NaN
       1986-02-28  1.196000e+04      NaN     NaN  NaN      NaN
       1986-03-31  1.633000e+04      NaN     NaN  NaN      NaN
       1986-04-30  1.517200e+04      NaN     NaN  NaN      NaN
       1986-05-31  1.179388e+04      NaN     NaN  NaN      NaN
...                         ...      ...     ...  ...      ...
93436  2024-08-31  6.840044e+08      NaN     NaN  NaN      NaN
       2024-09-30  8.390474e+08      NaN     NaN  NaN      NaN
       2024-10-31  8.020335e+08      NaN     NaN  NaN      NaN
       2024-11-30  1.107984e+09      NaN     NaN  NaN      NaN
       2024-12-31  1.296351e+09  72913.0  7091.0  0.0  97690.0

[3788855 rows x 5 columns]

In [9]:
# lag Market Cap by 6 months
ratios_df['Market Cap'] = ratios_df.groupby('PERMNO')['Market Cap'].shift(6)
# lag SEQ by 6 months
ratios_df['SEQ'] = ratios_df.groupby('PERMNO')['SEQ'].shift(6)
# lag IB by 6 months
ratios_df['IB'] = ratios_df.groupby('PERMNO')['IB'].shift(6)
# lag DVT by 6 months
ratios_df['DVT'] = ratios_df.groupby('PERMNO')['DVT'].shift(6)
# lag SALE by 6 months
ratios_df['SALE'] = ratios_df.groupby('PERMNO')['SALE'].shift(6)
ratios_df   

Market Cap  SEQ  IB  DVT  SALE
PERMNO DATE                                        
10000  1986-01-31           NaN  NaN NaN  NaN   NaN
       1986-02-28           NaN  NaN NaN  NaN   NaN
       1986-03-31           NaN  NaN NaN  NaN   NaN
       1986-04-30           NaN  NaN NaN  NaN   NaN
       1986-05-31           NaN  NaN NaN  NaN   NaN
...                         ...  ...  ..  ...   ...
93436  2024-08-31  6.429454e+08  NaN NaN  NaN   NaN
       2024-09-30  5.605883e+08  NaN NaN  NaN   NaN
       2024-10-31  5.845158e+08  NaN NaN  NaN   NaN
       2024-11-30  5.679320e+08  NaN NaN  NaN   NaN
       2024-12-31  6.321554e+08  NaN NaN  NaN   NaN

[3788855 rows x 5 columns]

In [10]:
# fill forward the Compustat data so that each month in the resulting dataset will have a value for the Compustat variables
ratios_df = ratios_df.sort_values(['PERMNO','DATE'])
ratios_df['Market Cap'] = ratios_df.groupby('PERMNO')['Market Cap'].ffill()
ratios_df['SEQ'] = ratios_df.groupby('PERMNO')['SEQ'].ffill()
ratios_df['IB'] = ratios_df.groupby('PERMNO')['IB'].ffill()
ratios_df['DVT'] = ratios_df.groupby('PERMNO')['DVT'].ffill()
ratios_df['SALE'] = ratios_df.groupby('PERMNO')['SALE'].ffill()
ratios_df


Market Cap      SEQ       IB  DVT     SALE
PERMNO DATE                                                    
10000  1986-01-31           NaN      NaN      NaN  NaN      NaN
       1986-02-28           NaN      NaN      NaN  NaN      NaN
       1986-03-31           NaN      NaN      NaN  NaN      NaN
       1986-04-30           NaN      NaN      NaN  NaN      NaN
       1986-05-31           NaN      NaN      NaN  NaN      NaN
...                         ...      ...      ...  ...      ...
93436  2024-08-31  6.429454e+08  62634.0  14997.0  0.0  96773.0
       2024-09-30  5.605883e+08  62634.0  14997.0  0.0  96773.0
       2024-10-31  5.845158e+08  62634.0  14997.0  0.0  96773.0
       2024-11-30  5.679320e+08  62634.0  14997.0  0.0  96773.0
       2024-12-31  6.321554e+08  62634.0  14997.0  0.0  96773.0

[3788855 rows x 5 columns]

In [11]:
# drop rows with missing values
ratios_df.dropna(inplace=True)
ratios_df

Market Cap        SEQ         IB    DVT       SALE
PERMNO DATE                                                            
10000  1987-04-30  3.002344e+03      0.418     -0.730  0.000      1.026
       1987-05-31  3.182504e+03      0.418     -0.730  0.000      1.026
10001  1986-12-31  6.033125e+03      5.432      0.669  0.365     21.460
       1987-01-31  5.971562e+03      5.432      0.669  0.365     21.460
       1987-02-28  6.402500e+03      5.432      0.669  0.365     21.460
...                         ...        ...        ...    ...        ...
93436  2024-08-31  6.429454e+08  62634.000  14997.000  0.000  96773.000
       2024-09-30  5.605883e+08  62634.000  14997.000  0.000  96773.000
       2024-10-31  5.845158e+08  62634.000  14997.000  0.000  96773.000
       2024-11-30  5.679320e+08  62634.000  14997.000  0.000  96773.000
       2024-12-31  6.321554e+08  62634.000  14997.000  0.000  96773.000

[2846932 rows x 5 columns]

In [12]:
# define a function that will take the accounting column name as input and return a vector or series of monthly portfolio returns

def compute_monthly_portfolio_returns(accounting_col, ratios_df=ratios_df, stocks=stocks):
    # # create the ratio column
    # ratios_df['ratio'] = ratios_df[accounting_col] / ratios_df['Market Cap']
    # Compute 1st and 99th percentiles for each DATE
    quantiles = (
        ratios_df.groupby(level='DATE')[[accounting_col]]
        .quantile([0.01, 0.99])
        .unstack()               # turns into columns like ('SEQ', 0.01), ('SEQ', 0.99)
    )
    quantiles.columns = ['q01', 'q99']  # flatten MultiIndex columns

    # print(quantiles)

    # Now we need to merge the quantiles for each date for these variables back into the dataframe to use them as cutoffs for the windsorization

    ratios_df_new = ratios_df.reset_index().merge(quantiles, left_on='DATE', right_index=True, how='left')

    # print(ratios_df_new)

    # Now we can use the `.clip()` method to windsorize the accounting value to the values of their 1st and 99th percentiles on each date.

    ratios_df_new[accounting_col] = ratios_df_new[accounting_col].clip(lower=ratios_df_new['q01'], upper=ratios_df_new['q99'])

    # Drop the quantile columns and any other columns we don't need(keep only 'PERMNO','DATE',accounting_col)
    ratios_df_new = ratios_df_new[['PERMNO','DATE',accounting_col]].copy()
    # print(ratios_df_new)

    # first need to rank the stocks each month by that metric, and select the top 1000
    ratios_df_new['rank'] = ratios_df_new.groupby('DATE')[accounting_col].rank(method='first', ascending=False)
    top1000 = ratios_df_new[ratios_df_new['rank'] <= 1000].copy()
    # print(top1000)

    # Restore the index
    top1000 = top1000.set_index(['PERMNO','DATE'])
    # merge with stocks to get returns
    merged_df = top1000[[accounting_col]].merge(stocks[['RET']], how='left', left_index=True, right_index=True)
    # print(merged_df)
    # drop rows with missing returns
    merged_df.dropna(subset=['RET'], inplace=True)
    # calculate the weighted returns for the top 1000 stocks
    merged_df['weight'] = merged_df.groupby('DATE')[accounting_col].transform(lambda x: x / x.sum())
    # use the previous month's weights to calculate the portfolio returns
    merged_df['weight'] = merged_df.groupby('PERMNO')['weight'].shift(1)
    merged_df.dropna(subset=['weight'], inplace=True)
    portfolio_returns = merged_df.groupby('DATE').apply(lambda x: (x['RET'] * x['weight']).sum())
    # print(portfolio_returns) 
    return portfolio_returns


In [13]:
Monthly_portfolio_returns_marketcap = compute_monthly_portfolio_returns('Market Cap')
Monthly_portfolio_returns_marketcap

DATE
1962-03-31    0.044581
1962-04-30   -0.025806
1962-05-31   -0.146515
1962-06-30   -0.047863
1962-07-31    0.075339
                ...   
2024-08-31    0.023756
2024-09-30    0.018872
2024-10-31   -0.009720
2024-11-30    0.068773
2024-12-31   -0.054450
Length: 754, dtype: float64

In [14]:
Monthly_portfolio_returns_SEQ = compute_monthly_portfolio_returns('SEQ')
Monthly_portfolio_returns_SEQ

DATE
1962-03-31    0.044581
1962-04-30   -0.025895
1962-05-31   -0.135900
1962-06-30   -0.063970
1962-07-31    0.070466
                ...   
2024-08-31    0.011751
2024-09-30    0.014159
2024-10-31   -0.007173
2024-11-30    0.075027
2024-12-31   -0.061652
Length: 754, dtype: float64

In [15]:
Monthly_portfolio_returns_IB = compute_monthly_portfolio_returns('IB')
Monthly_portfolio_returns_IB

DATE
1962-03-31    0.044581
1962-04-30   -0.025863
1962-05-31   -0.143535
1962-06-30   -0.036366
1962-07-31    0.071787
                ...   
2024-08-31    0.017158
2024-09-30    0.015990
2024-10-31   -0.006083
2024-11-30    0.071424
2024-12-31   -0.060396
Length: 754, dtype: float64

In [16]:
Monthly_portfolio_returns_DVT = compute_monthly_portfolio_returns('DVT')
Monthly_portfolio_returns_DVT

DATE
1962-03-31    0.044581
1962-04-30   -0.025842
1962-05-31   -0.143495
1962-06-30   -0.044612
1962-07-31    0.071503
                ...   
2024-08-31    0.024423
2024-09-30    0.016749
2024-10-31   -0.006918
2024-11-30    0.059891
2024-12-31   -0.063079
Length: 754, dtype: float64

In [17]:
Monthly_portfolio_returns_SALE = compute_monthly_portfolio_returns('SALE')
Monthly_portfolio_returns_SALE

DATE
1962-03-31    0.044581
1962-04-30   -0.026027
1962-05-31   -0.119910
1962-06-30   -0.134341
1962-07-31    0.065650
                ...   
2024-08-31    0.010232
2024-09-30    0.015574
2024-10-31   -0.014293
2024-11-30    0.073757
2024-12-31   -0.060149
Length: 754, dtype: float64

## **Question 1**
Create Table 1 from 1962 through 2004

In [18]:
def summarize_single_portfolio(name, returns, ref_returns):
    """
    Compute performance stats for a single portfolio vs Reference.

    Parameters
    ----------
    name : str
        Portfolio name, e.g. "Book" or "Sales".
    returns : pd.Series
        Monthly return series for this portfolio.
    ref_returns : pd.Series
        Monthly return series for the Reference portfolio.

    Returns
    -------
    pd.DataFrame
        One-row DataFrame with metrics for this portfolio.
    """

    MONTHS_PER_YEAR = 12
    ANNUALIZE = np.sqrt(MONTHS_PER_YEAR)

    # Drop missing months (align both)
    df = pd.concat([returns, ref_returns], axis=1).dropna()
    portfolio = df.iloc[:, 0]
    ref = df.iloc[:, 1]

    # Mean and volatility
    mean_monthly = portfolio.mean()
    vol_monthly = portfolio.std()

    ann_return = mean_monthly * MONTHS_PER_YEAR * 100
    ann_vol = vol_monthly * ANNUALIZE * 100

    # Sharpe ratio (without risk-free rate)
    sharpe = (mean_monthly / vol_monthly) * ANNUALIZE

    # Excess return vs Reference
    if name.lower() == "reference":
        excess_ret, t_stat = np.nan, np.nan
    else:
        diff = portfolio - ref
        excess_ret = diff.mean() * MONTHS_PER_YEAR * 100
        t_stat = diff.mean() / diff.std() * np.sqrt(len(diff))

    result = pd.DataFrame({
        "Portfolio": [name],
        "Ann. Return (%)": [round(ann_return, 2)],
        "Ann. Vol (%)": [round(ann_vol, 2)],
        "Sharpe": [round(sharpe, 2)],
        "Excess Return vs Ref (%)": [round(excess_ret, 2)],
        "t-stat": [round(t_stat, 2)]
    })

    return result


In [19]:
# keep the returns ONLY from 1962 through 2004
ref = Monthly_portfolio_returns_marketcap.loc['1962-01-31':'2004-12-31']
book = Monthly_portfolio_returns_SEQ.loc['1962-01-31':'2004-12-31']
income = Monthly_portfolio_returns_IB.loc['1962-01-31':'2004-12-31']
sales = Monthly_portfolio_returns_SALE.loc['1962-01-31':'2004-12-31']
dividends = Monthly_portfolio_returns_DVT.loc['1962-01-31':'2004-12-31']

t1_ref = summarize_single_portfolio("Reference", ref, ref)
t1_book = summarize_single_portfolio("Book", book, ref)
t1_income = summarize_single_portfolio("Income", income, ref)
t1_sales = summarize_single_portfolio("Sales", sales, ref)
t1_dividends = summarize_single_portfolio("Dividends", dividends, ref)
t1 = pd.concat([t1_ref, t1_book, t1_income, t1_sales, t1_dividends], ignore_index=True)
print(t1)

   Portfolio  Ann. Return (%)  Ann. Vol (%)  Sharpe  Excess Return vs Ref (%)  \
0  Reference            11.43         16.18    0.71                       NaN   
1       Book            12.57         15.75    0.80                      1.14   
2     Income            12.70         15.30    0.83                      1.27   
3      Sales            13.34         16.56    0.81                      1.91   
4  Dividends            12.48         14.40    0.87                      1.05   

   t-stat  
0     NaN  
1    2.15  
2    2.47  
3    2.74  
4    1.39  


All fundamental portfolios outperformed the reference (market-cap-weighted) portfolio by 1–2% per year.

Sharpe ratios were consistently higher (≈0.8 vs 0.7), showing improved risk-adjusted performance.

t-stats around 2–3 indicate that the alpha was statistically significant — this is exactly what the original Fundamental Indexation paper found.

# **Question 2** 
Produce a version of Table 1 from the paper, as in Question 1, but this time for the period 2005-2024. 

In [20]:
# keep the returns ONLY from 2005 through 2024
ref_out_of_sample = Monthly_portfolio_returns_marketcap.loc['2005-01-31':'2024-12-31']
book_out_of_sample = Monthly_portfolio_returns_SEQ.loc['2005-01-31':'2024-12-31']
income_out_of_sample = Monthly_portfolio_returns_IB.loc['2005-01-31':'2024-12-31']
sales_out_of_sample = Monthly_portfolio_returns_SALE.loc['2005-01-31':'2024-12-31']
dividends_out_of_sample = Monthly_portfolio_returns_DVT.loc['2005-01-31':'2024-12-31']

t1_ref_oos = summarize_single_portfolio("Reference", ref_out_of_sample, ref_out_of_sample)
t1_book_oos = summarize_single_portfolio("Book", book_out_of_sample, ref_out_of_sample)
t1_income_oos = summarize_single_portfolio("Income", income_out_of_sample, ref_out_of_sample)
t1_sales_oos = summarize_single_portfolio("Sales", sales_out_of_sample, ref_out_of_sample)
t1_dividends_oos = summarize_single_portfolio("Dividends", dividends_out_of_sample, ref_out_of_sample)
t1_oos = pd.concat([t1_ref_oos, t1_book_oos, t1_income_oos, t1_sales_oos, t1_dividends_oos], ignore_index=True)
print(t1_oos)


   Portfolio  Ann. Return (%)  Ann. Vol (%)  Sharpe  Excess Return vs Ref (%)  \
0  Reference            10.99         16.15    0.68                       NaN   
1       Book            10.82         17.86    0.61                     -0.17   
2     Income            11.20         16.93    0.66                      0.21   
3      Sales            12.16         18.27    0.67                      1.16   
4  Dividends            11.04         16.34    0.68                      0.05   

   t-stat  
0     NaN  
1   -0.19  
2    0.35  
3    1.27  
4    0.05  


when extended to 2005–2024, average return levels dropped slightly, excess returns diminished, Sharpe ratios fell, and most alphas lost significance.
This suggests that the original premium documented in the paper did not hold up well out-of-sample, likely due to factor crowding, valuation compression, or increased market efficiency.

# **Question 3** 
Create a version of Table 2, do this for both the 1962-2004 period and the 2005-present period.

In [33]:
# Read fama french file
import pandas as pd
ff_factors = pd.read_csv('FamaFrenchMonthly_HW4.csv')
ff_factors['Date'] = pd.to_datetime(ff_factors['Date'], format = '%Y%m')
ff_factors.set_index('Date', inplace = True)

#align the date to the end of the month
ff_factors.index = ff_factors.index.to_period('M').to_timestamp('M')
print(ff_factors.head())

            Mkt-RF   SMB   HML    RF
Date                                
1926-07-31    2.89 -2.55 -2.39  0.22
1926-08-31    2.64 -1.14  3.81  0.25
1926-09-30    0.38 -1.36  0.05  0.23
1926-10-31   -3.27 -0.14  0.82  0.32
1926-11-30    2.54 -0.11 -0.61  0.31


In [42]:
def summarize_capm_portfolio(name, returns2, ff_factors):
    
    import statsmodels.api as sm
    #combine portfolio return with fama franch market factor
    df2 = pd.concat([returns2, ff_factors[['Mkt-RF', 'RF']]], axis = 1).dropna()
    #convert % to decimal
    df2[['Mkt-RF', 'RF']] = df2[['Mkt-RF', 'RF']]/100

    df2['excess_ret']= df2.iloc[:,0] - df2['RF']
    df2['mkt_excess'] = df2['Mkt-RF']

    #CAPM model: Rp-Rf = alpha + beta*(mkt-Rf), then we get estimated alpha, and estimated beta
    x = sm.add_constant(df2['Mkt-RF'])
    y= df2['excess_ret']
    model = sm.OLS(y,x).fit()
    
    alpha_month = model.params['const']
    beta = model.params['Mkt-RF']
    t_alpha = model.tvalues['const']

    #Annualization
    ann_return = df2.iloc[:,0].mean()*12*100
    alpha_ann = alpha_month*12*100

    result2 = pd.DataFrame({
        "portfolio": [name],
        "Ann. Return (%)": [round(ann_return, 2)],
        "CAPM Beta": [round(beta,2)],
        "CAPM Alpha": [round(alpha_ann,2)],
        "t stat": [round(t_alpha,2)]})
    return result2
    

In [43]:
# keep the returns ONLY from 1962 through 2004
ref = Monthly_portfolio_returns_marketcap.loc['1962-01-31':'2004-12-31']
book = Monthly_portfolio_returns_SEQ.loc['1962-01-31':'2004-12-31']
income = Monthly_portfolio_returns_IB.loc['1962-01-31':'2004-12-31']
sales = Monthly_portfolio_returns_SALE.loc['1962-01-31':'2004-12-31']
dividends = Monthly_portfolio_returns_DVT.loc['1962-01-31':'2004-12-31']

t2_ref = summarize_capm_portfolio("Reference", ref, ff_factors)
t2_book = summarize_capm_portfolio("Book", book, ff_factors)
t2_income = summarize_capm_portfolio("Income", income, ff_factors)
t2_sales = summarize_capm_portfolio("Sales", sales, ff_factors)
t2_dividends = summarize_capm_portfolio("Dividends", dividends, ff_factors)
t2 = pd.concat([t2_ref, t2_book, t2_income, t2_sales, t2_dividends], ignore_index=True)
print(t2)

   portfolio  Ann. Return (%)  CAPM Beta  CAPM Alpha  t stat
0  Reference            11.43       1.02        0.10    0.21
1       Book            12.57       0.96        1.56    2.06
2     Income            12.70       0.94        1.83    2.55
3      Sales            13.34       1.00        2.10    2.47
4  Dividends            12.48       0.85        2.07    2.43


In [44]:
# keep the returns ONLY from 2005 through 2024
ref = Monthly_portfolio_returns_marketcap.loc['2005-01-31':'2024-12-31']
book = Monthly_portfolio_returns_SEQ.loc['2005-01-31':'2024-12-31']
income = Monthly_portfolio_returns_IB.loc['2005-01-31':'2024-12-31']
sales = Monthly_portfolio_returns_SALE.loc['2005-01-31':'2024-12-31']
dividends = Monthly_portfolio_returns_DVT.loc['2005-01-31':'2024-12-31']

t2_ref_oos = summarize_capm_portfolio("Reference", ref, ff_factors)
t2_book_oos = summarize_capm_portfolio("Book", book, ff_factors)
t2_income_oos = summarize_capm_portfolio("Income", income, ff_factors)
t2_sales_oos = summarize_capm_portfolio("Sales", sales, ff_factors)
t2_dividends_oos = summarize_capm_portfolio("Dividends", dividends, ff_factors)
t2_oos = pd.concat([t2_ref_oos, t2_book_oos, t2_income_oos, t2_sales_oos, t2_dividends_oos], ignore_index=True)
print(t2_oos)

   portfolio  Ann. Return (%)  CAPM Beta  CAPM Alpha  t stat
0  Reference            10.99       1.03       -0.55   -1.01
1       Book            10.82       1.10       -1.41   -1.17
2     Income            11.20       1.06       -0.62   -0.65
3      Sales            12.16       1.13       -0.32   -0.26
4  Dividends            11.04       1.00       -0.17   -0.14


The CAPM analysis shows that accounting-based factors generated significant abnormal returns before 2005 but lost their predictive power in the modern, more efficient market environment.

# **Question 4** 
Create table 3 including SMB factor and HML factor, and do this for both the 1962-2004 period and the 2005-present period.

In [50]:
def summarize_fama_portfolio(name, returns3, ff_factors):
    
    import statsmodels.api as sm
    #combine portfolio return with fama franch factors
    df3 = pd.concat([returns3, ff_factors[['Mkt-RF', 'RF', 'SMB', 'HML']]], axis = 1).dropna()
    #convert % to decimal
    df3[['Mkt-RF','SMB', 'HML', 'RF']] = df3[['Mkt-RF','SMB', 'HML','RF']]/100
    #Excess return
    df3['excess_ret']= df3.iloc[:,0] - df3['RF']

    #CAPM model: Rp-Rf = alpha + beta_m*(mkt-Rf)+ beta_s * SMB+ beta_h * HML
    x = sm.add_constant(df3[['Mkt-RF','SMB', 'HML']])                     
    y= df3['excess_ret']
    model = sm.OLS(y,x).fit()
    
    alpha_month = model.params['const']
    beta_m = model.params['Mkt-RF']
    beta_s = model.params['SMB']
    beta_h = model.params['HML']
    t_alpha = model.tvalues['const']

    #Annualization
    ann_return = df3.iloc[:,0].mean()*12*100
    alpha_ann = alpha_month*12*100

    result3 = pd.DataFrame({
        "portfolio": [name],
        "Ann. Return (%)": [round(ann_return, 2)],
        "Beta_Mkt": [round(beta_m,2)],
        "Beta_SMB": [round(beta_s,2)],
        "Beta_HML": [round(beta_h,2)],
        "CAPM Alpha": [round(alpha_ann,2)],
        "t stat": [round(t_alpha,2)]})
    return result3
    

In [51]:
# keep the returns ONLY from 1962 through 2004
ref = Monthly_portfolio_returns_marketcap.loc['1962-01-31':'2004-12-31']
book = Monthly_portfolio_returns_SEQ.loc['1962-01-31':'2004-12-31']
income = Monthly_portfolio_returns_IB.loc['1962-01-31':'2004-12-31']
sales = Monthly_portfolio_returns_SALE.loc['1962-01-31':'2004-12-31']
dividends = Monthly_portfolio_returns_DVT.loc['1962-01-31':'2004-12-31']

t3_ref = summarize_fama_portfolio("Reference", ref, ff_factors)
t3_book = summarize_fama_portfolio("Book", book, ff_factors)
t3_income = summarize_fama_portfolio("Income", income, ff_factors)
t3_sales = summarize_fama_portfolio("Sales", sales, ff_factors)
t3_dividends = summarize_fama_portfolio("Dividends", dividends, ff_factors)
t3 = pd.concat([t3_ref, t3_book, t3_income, t3_sales, t3_dividends], ignore_index=True)
print(t3)

   portfolio  Ann. Return (%)  Beta_Mkt  Beta_SMB  Beta_HML  CAPM Alpha  \
0  Reference            11.43      1.04     -0.06      0.05       -0.13   
1       Book            12.57      1.05     -0.01      0.35       -0.92   
2     Income            12.70      1.01     -0.05      0.29       -0.13   
3      Sales            13.34      1.08      0.10      0.39       -0.81   
4  Dividends            12.48      0.96     -0.07      0.40       -0.65   

   t stat  
0   -0.27  
1   -1.55  
2   -0.21  
3   -1.19  
4   -1.02  


In [53]:
# keep the returns ONLY from 2005 through 2024
ref = Monthly_portfolio_returns_marketcap.loc['2005-01-31':'2024-12-31']
book = Monthly_portfolio_returns_SEQ.loc['2005-01-31':'2024-12-31']
income = Monthly_portfolio_returns_IB.loc['2005-01-31':'2024-12-31']
sales = Monthly_portfolio_returns_SALE.loc['2005-01-31':'2024-12-31']
dividends = Monthly_portfolio_returns_DVT.loc['2005-01-31':'2024-12-31']

t3_ref_oos = summarize_fama_portfolio("Reference", ref, ff_factors)
t3_book_oos = summarize_fama_portfolio("Book", book, ff_factors)
t3_income_oos = summarize_fama_portfolio("Income", income, ff_factors)
t3_sales_oos = summarize_fama_portfolio("Sales", sales, ff_factors)
t3_dividends_oos = summarize_fama_portfolio("Dividends", dividends, ff_factors)
t3_oos = pd.concat([t3_ref_oos, t3_book_oos, t3_income_oos, t3_sales_oos, t3_dividends_oos], ignore_index=True)
print(t3_oos)

   portfolio  Ann. Return (%)  Beta_Mkt  Beta_SMB  Beta_HML  CAPM Alpha  \
0  Reference            10.99      1.02      0.03      0.09       -0.29   
1       Book            10.82      1.04      0.12      0.37       -0.36   
2     Income            11.20      1.03      0.04      0.26        0.04   
3      Sales            12.16      1.07      0.15      0.34        0.72   
4  Dividends            11.04      0.96      0.01      0.37        0.65   

   t stat  
0   -0.59  
1   -0.51  
2    0.06  
3    0.86  
4    0.82  


# **Question 5** 

1. How do the CAPM results change out-of-sample?

   In the 1962–2004 sample, the CAPM alphas were mostly small and negative, In the out-of-sample period (2005–2024), alphas became slightly higher and     even positive for some portfolios, but none were statistically significant
   
2. What happens to the CAPM results in the paper's sample once we include other factors in the FF3 model?
   
   Adding SMB and HML does not eliminate alpha completely; some alphas even increase slightly in the post-2005 period.
   This suggests that the traditional size and value factors no longer fully capture modern market risk sources such as growth, profitability, and         intangible assets.
   Hence, the FF3 model improves fit in the early sample but loses explanatory power in the recent sample, causing residual returns to rise slightly.

3. What does this imply about the source of return being captured by the factors book, income, sales, and dividends?

   These fundamental portfolios earn their returns mainly through systematic factor exposures rather than mispricing.
   In the 1962–2004 sample, their positive HML betas indicate strong exposure to value risk and help explain their higher average returns.
   In the 2005–2024 period, HML effects remain positive but weaker, and SMB turns slightly positive, signaling greater influence from size and growth      effects.
   Thus, their performance is factor-driven and time-varying. 
  
7. What has this analysis taught us about the nature of returns to “Fundamental Indexation”?

   Fundamental indexing does not deliver persistent abnormal returns after controlling for risk factors.
   Its historical outperformance in the 1960s–1990s reflects a value tilt (HML exposure), while in recent years the same tilt has lost effect as the       value premium declined.
   Therefore, fundamental indexation mainly offers alternative factor exposure rather than true alpha generation, it is a risk-tilted strategy, not a       source of free excess return.


### **Conclusion**
- The CAPM explains returns well in early sample, but loses power later.
- Fama-French factors (SMB, HML) reduce alpha but do not eliminate it.
- Fundamental indexation provides value exposure, not true alpha.
